In [3]:
import pandas as pd
import numpy as np
import requests
import json
import gzip
import zipfile
from datetime import datetime

In [4]:
minfo=pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep='\t')
reviews=pd.read_csv('zippedData/rt.reviews.tsv.gz',sep ='\t',encoding='latin1')

# Inspecting  - Movie Information List Cleaning!

In [5]:
#A basic overview of the first 5 rows!
minfo.head(10)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
5,8,The year is 1942. As the Allies unite overseas...,PG,Drama|Kids and Family,Jay Russell,Gail Gilchriest,"Mar 3, 2000","Jul 11, 2000",NaN,NaN,95 minutes,Warner Bros. Pictures
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,"41,032,915",82 minutes,Paramount Pictures
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,"224,114",123 minutes,Sony Pictures Classics
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,"134,904",117 minutes,NaN
9,15,When a diamond expedition in the Congo is lost...,PG-13,Action and Adventure|Mystery and Suspense|Scie...,Frank Marshall,John Patrick Shanley,"Jun 9, 1995","Jul 27, 1999",NaN,NaN,108 minutes,NaN


In [8]:
minfo['synopsis'][1]

"New York City, not-too-distant-future: Eric Packer, a 28 year-old finance golden boy dreaming of living in a civilization ahead of this one, watches a dark shadow cast over the firmament of the Wall Street galaxy, of which he is the uncontested king. As he is chauffeured across midtown Manhattan to get a haircut at his father's old barber, his anxious eyes are glued to the yuan's exchange rate: it is mounting against all expectations, destroying Eric's bet against it. Eric Packer is losing his empire with every tick of the clock. Meanwhile, an eruption of wild activity unfolds in the city's streets. Petrified as the threats of the real world infringe upon his cloud of virtual convictions, his paranoia intensifies during the course of his 24-hour cross-town odyssey. Packer starts to piece together clues that lead him to a most terrifying secret: his imminent assassination. -- (C) Official Site"

In [379]:
minfo.dtypes

id               int64
synopsis        object
rating          object
genre           object
director        object
writer          object
theater_date    object
dvd_date        object
currency        object
box_office      object
runtime         object
studio          object
dtype: object

In [380]:
#Checking NA's in files
minfo.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [381]:
minfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [382]:
#Dropping all columns both irrelevant and littered with NA's
minfo.drop(columns = ['synopsis', 'currency', 'dvd_date', 'theater_date'], inplace=True )

In [383]:
minfo_boxoffice= minfo['box_office']
minfo_boxoffice.sort_values()

86      1,000,000
15      1,039,869
809     1,110,286
1087    1,121,000
1213    1,149,350
          ...    
1554          NaN
1556          NaN
1557          NaN
1558          NaN
1559          NaN
Name: box_office, Length: 1560, dtype: object

In [386]:
minfo.dropna()

,id,rating,genre,director,writer,box_office,runtime,studio
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"600,000",108 minutes,Entertainment One
6,10,PG-13,Comedy,Jake Kasdan,Mike White,"41,032,915",82 minutes,Paramount Pictures
7,13,R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"224,114",123 minutes,Sony Pictures Classics
15,22,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,"1,039,869",108 minutes,ATO Pictures
18,25,PG-13,Action and Adventure|Drama|Science Fiction and...,Carl Erik Rinsch,Chris Morgan|Hossein Amini,"20,518,224",127 minutes,Universal Pictures
...,...,...,...,...,...,...,...,...
1530,1968,PG,Comedy|Drama,Cameron Crowe,Aline Brosh McKenna|Cameron Crowe,"72,700,000",126 minutes,20th Century Fox
1537,1976,NR,Action and Adventure|Art House and International,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,"1,320,005",123 minutes,Buffalo Films
1541,1980,PG-13,Action and Adventure|Science Fiction and Fantasy,Joss Whedon,Joss Whedon,"25,335,935",119 minutes,Universal Pictures
1542,1981,NR,Comedy|Drama,Gauri Shinde,Gauri Shinde,"1,416,189",129 minutes,Eros Entertainment


# Looking at Movie Info

In [387]:
minfo['rating'].value_counts()

R        521
NR       503
PG       240
PG-13    235
G         57
NC17       1
Name: rating, dtype: int64

In [388]:
m_box = minfo['box_office'].dropna() #Sort this
m_box

1          600,000
6       41,032,915
7          224,114
8          134,904
15       1,039,869
           ...    
1541    25,335,935
1542     1,416,189
1545        59,371
1546       794,306
1555    33,886,034
Name: box_office, Length: 340, dtype: object

1. We Want to consider relationship between run time and box office
2. Relationship between rating and box office
3. find similar writers and directors and if larger correlation


In [389]:
m_dir = minfo['director']
m_dir.value_counts()

Steven Spielberg    10
Clint Eastwood       8
Alfred Hitchcock     4
Curtis Hanson        4
William Friedkin     4
                    ..
John Maybury         1
Bob Badway           1
Michael Rhodes       1
Heiner Carow         1
David Koepp          1
Name: director, Length: 1125, dtype: int64

In [390]:
m_writ = minfo['writer']
m_writ.value_counts()

Woody Allen                      4
Jim Jarmusch                     3
John Hughes                      3
Sylvester Stallone               3
Hong Sang-soo                    3
                                ..
Zak Penn|Ernest Cline            1
Eric Rohmer                      1
Paul Pender                      1
Shawn Christensen|Jason Dolan    1
David Koepp                      1
Name: writer, Length: 1069, dtype: int64

In [391]:
minfo['runtime'].value_counts()

90 minutes     72
95 minutes     66
100 minutes    51
93 minutes     47
96 minutes     43
               ..
171 minutes     1
64 minutes      1
159 minutes     1
183 minutes     1
166 minutes     1
Name: runtime, Length: 142, dtype: int64

In [392]:
m_genre = minfo['genre']
m_genre.value_counts()

#parsing through the mult titles

Drama                                                                                                                151
Comedy                                                                                                               110
Comedy|Drama                                                                                                          80
Drama|Mystery and Suspense                                                                                            67
Art House and International|Drama                                                                                     62
                                                                                                                    ... 
Classics|Documentary|Drama|Romance                                                                                     1
Action and Adventure|Art House and International|Drama|Faith and Spirituality                                          1
Drama|Science Fiction and Fantas

In [393]:
minfo.head()

,id,rating,genre,director,writer,box_office,runtime,studio
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,NaN,104 minutes,NaN
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"600,000",108 minutes,Entertainment One
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,NaN,116 minutes,NaN
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,NaN,128 minutes,NaN
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,200 minutes,NaN


# Cleaning Movie Info

In [394]:
minfo['writer'] = minfo['writer'].str.replace('|', ' ')

In [395]:
minfo

,id,rating,genre,director,writer,box_office,runtime,studio
0,1,R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,NaN,104 minutes,NaN
1,3,R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg Don DeLillo,"600,000",108 minutes,Entertainment One
2,5,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,NaN,116 minutes,NaN
3,6,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio Michael Crichton,NaN,128 minutes,NaN
4,7,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...
1555,1996,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"33,886,034",106 minutes,New Line Cinema
1556,1997,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner Tom Davis Dan Aykroyd Bonnie Turner,NaN,88 minutes,Paramount Vantage
1557,1998,G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,NaN,111 minutes,NaN
1558,1999,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans Robert Gunter,NaN,101 minutes,NaN


# Inspecting at Reviews

In [396]:
reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [397]:
reviews.dtypes

id             int64
review        object
rating        object
fresh         object
critic        object
top_critic     int64
publisher     object
date          object
dtype: object

In [398]:
reviews.isna().sum()

id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64

In [399]:
reviews.drop(columns = ['rating', 'review', 'critic','top_critic'], inplace=True)

In [400]:
reviews.info

<bound method DataFrame.info of          id   fresh         publisher                date
0         3   fresh   Patrick Nabarro   November 10, 2018
1         3  rotten           io9.com        May 23, 2018
2         3   fresh  Stream on Demand     January 4, 2018
3         3   fresh              MUBI   November 16, 2017
4         3   fresh      Cinema Scope    October 12, 2017
...     ...     ...               ...                 ...
54427  2000   fresh     Village Voice  September 24, 2002
54428  2000  rotten        Zap2it.com  September 21, 2005
54429  2000  rotten   EmanuelLevy.Com       July 17, 2005
54430  2000  rotten    Filmcritic.com   September 7, 2003
54431  2000   fresh      Showbizz.net   November 12, 2002

[54432 rows x 4 columns]>

In [401]:
reviews.head()

,id,fresh,publisher,date
0,3,fresh,Patrick Nabarro,"November 10, 2018"
1,3,rotten,io9.com,"May 23, 2018"
2,3,fresh,Stream on Demand,"January 4, 2018"
3,3,fresh,MUBI,"November 16, 2017"
4,3,fresh,Cinema Scope,"October 12, 2017"


In [311]:
m_genre.describe()

count      1552
unique      299
top       Drama
freq        151
Name: genre, dtype: object

# Looking at Reviews

In [315]:
pub_review = reviews['publisher']
pub_review.value_counts()

eFilmCritic.com                    673
EmanuelLevy.Com                    591
New York Times                     590
Washington Post                    565
Entertainment Weekly               541
                                  ... 
Movie Guys with Jeff &amp; Dave      1
The Jacobin                          1
Nerve.com                            1
CineGods.com                         1
Elements of Madness                  1
Name: publisher, Length: 1281, dtype: int64

In [317]:
fresh_review = reviews['fresh']
fresh_review.value_counts()

fresh     33035
rotten    21397
Name: fresh, dtype: int64